# CCSD theory for a closed-shell reference

In this notebook we will use wicked to generate and implement equations for the CCSD method.
To simplify this notebook some of the utility functions are imported from the file `examples_helpers.py`.
In this example, we run a CCSD computation on the H<sub>6</sub> molecule, reading all the relevant information from the file `sr-h6-sto-3g.npy`.

In [1]:
import time
import wicked as w
import numpy as np
from examples_helpers import *

## Read calculation information (integrals, number of orbitals)

We start by reading information about the reference state, integrals, and denominators from the file `sr-h6-sto-3g.npy`. The variable `H` is a dictionary that holds the blocks of the Hamiltonian **normal-ordered** with respect to the Hartree–Fock determinant. `invD` similarly is a dictionary that stores the denominators $(\epsilon_i + \epsilon_j + \ldots - \epsilon_a - \epsilon_b - \ldots)^{-1}$.

In [2]:
molecule = "sr-h6-sto-3g"

with open(f'{molecule}.npy', 'rb') as f:
    Eref = np.load(f)
    nocc, nvir = np.load(f)
    H = np.load(f, allow_pickle=True).item()

invD = compute_inverse_denominators(H,nocc,nvir,2)

## Compute the Hartree–Fock and MP2 energy

To verify that the Hamiltonian is read correctly, we compute the MP2 correlation energy

In [3]:
# Compute the MP2 correlation energy
Emp2 = 0.0
for i in range(nocc):
    for j in range(nocc):
        for a in range(nvir):
            for b in range(nvir):
                Emp2 += 0.25 * H["oovv"][i][j][a][b] ** 2 * invD["oovv"][i][j][a][b]

print(f"MP2 correlation energy: {Emp2:.12f} Eh")

MP2 correlation energy: -0.066236921094 Eh


## Define orbital spaces and the Hamiltonian and cluster operators

Here we define the cluster operator (`Top`) and the Hamiltonian (`Hop`) that will be used to derive the CCSD equations. We also define the similarity-transformed Hamiltonian $\bar{H}$ truncated at the four-nested commutator:

\begin{equation}
\bar{H} = \hat{H} + [\hat{H},\hat{T}] + \frac{1}{2} [[\hat{H},\hat{T}],\hat{T}]
+ \frac{1}{6} [[[\hat{H},\hat{T}],\hat{T}],\hat{T}]
+ \frac{1}{24} [[[[\hat{H},\hat{T}],\hat{T}],\hat{T}],\hat{T}] + \ldots
\end{equation}

In [4]:
w.reset_space()
w.add_space("o", "fermion", "occupied", ["i", "j", "k", "l", "m", "n"])
w.add_space("v", "fermion", "unoccupied", ["a", "b", "c", "d", "e", "f"])

Top = w.op("T", ["v+ o", "v+ v+ o o"])
Hop = w.utils.gen_op("H",1,"ov","ov") + w.utils.gen_op("H",2,"ov","ov")
# the similarity-transformed Hamiltonian truncated to the four-nested commutator term
Hbar = w.bch_series(Hop,Top,4)

In the following lines, we apply Wick's theorem to simplify the similarity-transformed Hamiltonian $\bar{H}$ computing all contributions ranging from operator rank 0 to 4 (double substitutions).
Then we convert all the terms into many-body equations accumulated into the residual `R`.

In [5]:
wt = w.WickTheorem()
expr = wt.contract(w.rational(1), Hbar, 0, 4)
mbeq = expr.to_manybody_equation("R")

Here we finally generate the CCSD equations. We use the utility function `generate_equation` to extract the equations corresponding to a given number of creation and annihilation operators and generated Python functions that we then define with the command `exec`

In [6]:
energy_eq = generate_equation(mbeq, 0,0)
t1_eq = generate_equation(mbeq, 1,1)
t2_eq = generate_equation(mbeq, 2,2)

exec(energy_eq)
exec(t1_eq)
exec(t2_eq)

# show what do these functions look like
print(energy_eq)

def evaluate_residual_0_0(H,T):
    # contributions to the residual
    R = 0.0
    R += 1.000000000 * np.einsum("ai,ia->",H["vo"],T["ov"],optimize="optimal")
    R += 0.250000000 * np.einsum("abij,ijab->",H["vvoo"],T["oovv"],optimize="optimal")
    R += 0.500000000 * np.einsum("abij,jb,ia->",H["vvoo"],T["ov"],T["ov"],optimize="optimal")
    return R


## CCSD algorithm

Here we code a simple loop in which we evaluate the energy and residuals of the CCSD equations and update the amplitudes

In [7]:
Ecorr_ref  = -0.107582941213 # from psi4numpy (H6)

T = {"ov" : np.zeros((nocc,nvir)),
     "oovv" : np.zeros((nocc,nocc,nvir,nvir))}

header = "Iter.     Energy [Eh]    Corr. energy [Eh]       |R|       "
print("-" * len(header))
print(header)
print("-" * len(header))

start = time.perf_counter()

maxiter = 50

for i in range(maxiter):
    # 1. compute energy and residuals
    R = {}
    Ecorr_w = evaluate_residual_0_0(H,T)
    Etot_w = Eref + Ecorr_w
    R['ov'] = evaluate_residual_1_1(H,T)
    Roovv = evaluate_residual_2_2(H,T)
    R['oovv'] = antisymmetrize_residual_2_2(Roovv,nocc,nvir)

    # 2. amplitude update
    update_amplitudes_ccsd(T,R,invD)

    # 3. check for convergence
    norm_R = np.sqrt(np.linalg.norm(R['ov'])**2 + np.linalg.norm(R['oovv'])**2)
    print(f"{i:3d}    {Etot_w:+.12f}    {Ecorr_w:+.12f}    {norm_R:e}") 
    if norm_R < 1.0e-8:
        break
        
end = time.perf_counter()
t = end - start        
        
print("-" * len(header))    
print(f"CCSD total energy                   {Etot_w:+.12f} [Eh]")
print(f"CCSD correlation energy             {Ecorr_w:+.12f} [Eh]")
print(f"Reference CCSD correlation energy   {Ecorr_ref:+.12f} [Eh]")
print(f"Error                               {Ecorr_w - Ecorr_ref:+.12e} [Eh]")
print(f"Timing                              {t:+.12e} [s]")
assert np.isclose(Ecorr_w, Ecorr_ref)

-----------------------------------------------------------
Iter.     Energy [Eh]    Corr. energy [Eh]       |R|       
-----------------------------------------------------------
  0    -3.111429681878    +0.000000000000    7.023516e-01
  1    -3.177666602972    -0.066236921094    2.703474e-01
  2    -3.200376118309    -0.088946436431    1.217901e-01
  3    -3.210146292908    -0.098716611030    6.159876e-02
  4    -3.214486966896    -0.103057285019    3.406897e-02
  5    -3.216580748075    -0.105151066198    2.036844e-02
  6    -3.217630916990    -0.106201235112    1.283723e-02
  7    -3.218192853277    -0.106763171400    8.324145e-03
  8    -3.218507725256    -0.107078043378    5.488428e-03
  9    -3.218693412710    -0.107263730832    3.648039e-03
 10    -3.218806802192    -0.107377120315    2.437192e-03
 11    -3.218878180015    -0.107448498137    1.632438e-03
 12    -3.218923995682    -0.107494313804    1.095330e-03
 13    -3.218953847724    -0.107524165846    7.356232e-04
 14    -